In [11]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import warnings
from fake_useragent import UserAgent
import warnings
import warnings

class Webscraper():
    
    def __init__(self, urls):
        self.data = None
        self.urls = list(urls)
        self.smm_dict = None
         
    def scrap_smm_info_from_pages(self):

        social_media_patterns = ['www.facebook.com', 'www.instagram.com', 'www.linkedin.com', 'www.twitter.com',
                                  'facebook.com', 'instagram.com', 'linkedin.com', 'twitter.com', 'x.com']
        ua = UserAgent()
        fake_headers = ua.chrome
        smm_dict = {}
        warnings.filterwarnings("ignore")
        
        for id4, smm_find in enumerate(self.urls):
            smm_links = set()
            try:
                response2 = requests.get(smm_find, timeout=8, headers={'User-Agent' : fake_headers}, verify=False)
                soupy = BeautifulSoup(response2.text, 'html.parser')
                body = soupy.find('body')

                if body is not None:
                    href_values = [link.get('href') for link in body.find_all('a') if link.get('href')]

                    for href in href_values:
                        for pattern in social_media_patterns:
                            if pattern in href:
                                smm_links.add(href)

            except Exception as e:
                pass

            if smm_links:
                smm_dict[id4] = list(smm_links)

        self.smm_dict = smm_dict
        return smm_dict


class Parser(Webscraper):
    
    def create_df_cleaned_from_smm(self, smm_dict):
        df_of_smm_opt = pd.DataFrame(smm_dict.items(), columns=['ID', 'Social Media Links'])

        platforms = ['Facebook', 'Instagram', 'LinkedIn', 'Twitter']

        for platform in platforms:
            df_of_smm_opt[platform] = df_of_smm_opt['Social Media Links'].apply(
                lambda links: next((link for link in links if platform.lower() in link.lower()), None)
            )

        df_of_smm_opt = df_of_smm_opt.drop('Social Media Links', axis=1)

        return df_of_smm_opt


In [14]:
urls = ['https://familyfriendsvet.com', 'https://bluepearlvet.com', 'https://www.westmichiganaeh.com', 'https://willowrunvetservices.com']

scraper = Webscraper(urls)

result = scraper.scrap_smm_info_from_pages()
final = Parser(result)
df = final.create_df_cleaned_from_smm(result)
df

#clean up scrapper, and clean up the damn logic, this is not clean
# Make github push tomorrow, also youll watch vid to make lirary after oop is done

,ID,Facebook,Instagram,LinkedIn,Twitter
0,0,https://www.facebook.com/familyfriendsvet/,https://www.instagram.com/familyfriendsveterin...,None,None
1,1,https://www.facebook.com/BluePearlPetHospital/,https://www.instagram.com/bluepearlvet/,https://www.linkedin.com/company/bluepearl-vet...,https://twitter.com/bluepearlvet
2,2,https://www.facebook.com/AEHGR,None,None,None
3,3,https://www.facebook.com/Willow-Run-Veterinary...,None,None,None


In [ ]:
def create_df_cleaned_from_smm(smm_dict):  

    df_of_smm_opt = pd.DataFrame(smm_dict.items(), columns=['ID', 'Social Media Links'])
    
    platforms = ['Facebook', 'Instagram', 'LinkedIn','Twitter']

    for platform in platforms:
        df_of_smm_opt[platform] = df_of_smm_opt['Social Media Links'].apply(
            lambda links: [link for link in links if platform.lower() in link.lower()][0] if any(platform.lower() in link.lower() for link in links) else None
        )
    df_of_smm_opt = df_of_smm_opt.drop('Social Media Links', axis=1)
    
    return df_of_smm_opt

In [ ]:
if __name__ == '__main__':
    create_df_cleaned_from_smm(scrap_smm_info_from_pages(get_accounts_websites_for_smm_scrap()))

df_of_smm_opt

save = df_of_smm_opt.to_csv('smm_25k_homepage_scrapped.csv')